In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [11]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.

vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_Vacation.csv")
vacation_df.head()



,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Jamestown,US,78.03,overcast clouds,42.0970,-79.2353,Jamestown
1,4,Bani,DO,85.86,clear sky,18.2833,-70.3333,Baní
2,5,Cabo San Lucas,MX,87.10,clear sky,22.8909,-109.9124,Cabo San Lucas
3,7,Lorengau,PG,80.28,overcast clouds,-2.0226,147.2712,Lorengau
4,11,Gewane,ET,94.30,scattered clouds,10.1664,40.6453,Gewana


In [12]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [13]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [14]:
# Trip to Praia Da Vitoria.
# Grab the first hotel from the results and store the name.
pg_vacation_df = vacation_df.loc[(vacation_df["Country"] == "IT")]
pg_vacation_df.dropna()

new_column_order = ["City_ID", "City", "Country", "Lat","Lng", "Max Temp", "Current Description"]
pg_vacation_df = pg_vacation_df[new_column_order]
pg_vacation_df

,City_ID,City,Country,Lat,Lng,Max Temp,Current Description
76,174,Portoferraio,IT,42.8115,10.3146,93.72,clear sky
84,192,Lasa,IT,46.6166,10.7002,89.69,broken clouds
88,203,Myra,IT,45.4358,12.1348,98.15,clear sky
169,356,Guastalla,IT,44.9132,10.6609,100.62,clear sky
201,412,Trapani,IT,37.8333,12.6667,92.28,few clouds
249,513,Codogno,IT,45.1619,9.7022,99.63,clear sky
285,581,Mantua,IT,45.1667,10.7833,100.09,clear sky


In [15]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 

start = float(input("What is your Starting City_ID from MV List you would like for your trip? "))
stop1 = float(input("What is your First Stop City_ID from MV List you would like for your trip? "))
stop2 = float(input("What is your Second Stop City_ID from MV List you would like for your trip? "))
stop3 = float(input("What is your Thirth Stop City_ID from MV List you would like for your trip? "))
end = float(input("What is your Last City_ID from MV List you would like for your trip? "))

In [16]:
# Start Vacation at:

vacation_start = pg_vacation_df.loc[(pg_vacation_df["City_ID"] == stop1)]
vacation_start = vacation_start.iloc[0]

vacation_stop1 = pg_vacation_df.loc[(pg_vacation_df["City_ID"] == stop2)]
vacation_stop1 = vacation_stop1.iloc[0]

vacation_stop2 = pg_vacation_df.loc[(pg_vacation_df["City_ID"] == stop3)]
vacation_stop2 = vacation_stop2.iloc[0]

vacation_stop3 = pg_vacation_df.loc[(pg_vacation_df["City_ID"] == end)]
vacation_stop3 = vacation_stop3.iloc[0]

vacation_end = pg_vacation_df.loc[(pg_vacation_df["City_ID"] <= end)]
vacation_end = vacation_end.iloc[0] 
                                     
print("Your Starting Destination is " + vacation_start["City"] + ", " + vacation_start["Country"])
print("Your Last Stop in your vacation is " + vacation_end["City"] + ", " + vacation_end["Country"])
print("Your Frist Stop is " + vacation_stop1["City"] + ", " + vacation_stop1["Country"])
print("Your Second Stop is " + vacation_stop2["City"] + ", " + vacation_stop2["Country"])                                     
print("Your Thirth Stop is " + vacation_stop3["City"] + ", " + vacation_stop3["Country"])      

Your Starting Destination is Guastalla, IT
Your Last Stop in your vacation is Portoferraio, IT
Your Frist Stop is Mantua, IT
Your Second Stop is Myra, IT
Your Thirth Stop is Guastalla, IT


In [17]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[["Lat", "Lng"]]
print(start)

Lat    44.9132
Lng    10.6609
Name: 169, dtype: object


In [18]:
stop1 = vacation_stop1[["Lat", "Lng"]]
print(stop1)

Lat    45.1667
Lng    10.7833
Name: 285, dtype: object


In [19]:
stop2 = vacation_stop2[["Lat", "Lng"]]
print(stop2)

Lat    45.4358
Lng    12.1348
Name: 88, dtype: object


In [20]:
stop3 = vacation_stop3[["Lat", "Lng"]]
print(stop3)

Lat    44.9132
Lng    10.6609
Name: 169, dtype: object


In [21]:
end = vacation_end[["Lat", "Lng"]]
print(end)

Lat    42.8115
Lng    10.3146
Name: 76, dtype: object


In [22]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

import gmaps
import gmaps.datasets
gmaps.configure(api_key=g_key)

# start = (34.8791, -76.9013)
# stop1 = (39.1834, -84.5333)
# stop2 = (37.4636, -122.4286)
# stop3 = (39.5963, -119.776)
# end = (22.0752, -159.319)

fig = gmaps.figure()
sart2end_via_stops = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')
fig.add_layer(sart2end_via_stops)
fig

Figure(layout=FigureLayout(height='420px'))

In [91]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.

pg_vacation_df = pd.concat([pg_vacation_df],ignore_index=True)
pg_vacation_df

,City_ID,City,Country,Lat,Lng,Max Temp,Current Description
0,5,Praia Da Vitoria,PT,38.7333,-27.0667,79.05,broken clouds
1,93,Ponta Do Sol,PT,32.6667,-17.1000,77.13,scattered clouds
2,118,Peniche,PT,39.3558,-9.3811,75.29,clear sky
3,188,Lata,PT,40.1629,-8.3327,80.17,scattered clouds
4,261,Estremoz,PT,38.8443,-7.5859,94.03,broken clouds
5,282,Ribeira Grande,PT,38.5167,-28.7000,77.40,few clouds


In [92]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
mark_info = [info_box_template.format(**row) for index, row in pg_vacation_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = pg_vacation_df[["Lat", "Lng"]]

In [93]:
# 11a. Add a marker layer for each city to the map.
city_mark = pg_vacation_df["City"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=mark_info)

fig.add_layer(marker_layer)

In [94]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))